In [20]:
# Source code from: https://github.com/analyticswithadam/Python/blob/main/Pull_all_Comments_and_Replies_for_YouTube_Playlists.ipynb
from googleapiclient.discovery import build
import pandas as pd
import os
from load_dotenv import load_dotenv

In [21]:
load_dotenv()
video_id = os.getenv("YOUTUBE_VIDEO_ID")
api_key = os.getenv("GOOGLE_YOUTUBE_V3_API_KEY")

In [22]:
# youtube client
youtube = build('youtube', 'v3', developerKey=api_key)

In [23]:
# # Function to get replies for a specific comment
# def get_replies(youtube, parent_id, video_id):  # Added video_id as an argument
#     replies = []
#     next_page_token = None

#     while True:
#         reply_request = youtube.comments().list(
#             part="snippet",
#             parentId=parent_id,
#             textFormat="plainText",
#             maxResults=100,
#             pageToken=next_page_token
#         )
#         reply_response = reply_request.execute()

#         for item in reply_response['items']:
#             comment = item['snippet']
#             replies.append({
#                 'Timestamp': comment['publishedAt'],
#                 'Username': comment['authorDisplayName'],
#                 'VideoID': video_id,
#                 'Comment': comment['textDisplay'],
#                 'Date': comment['updatedAt'] if 'updatedAt' in comment else comment['publishedAt']
#             })

#         next_page_token = reply_response.get('nextPageToken')
#         if not next_page_token:
#             break

#     return replies


# Function to get all comments (including replies) for a single video
def get_comments_for_video(youtube, video_id):
    all_comments = []
    next_page_token = None

    while True:
        comment_request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100
        )
        comment_response = comment_request.execute()

        for item in comment_response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Comment': top_comment['textDisplay'],
            })

        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break

    return all_comments

# List to hold all comments from all videos
all_comments = []


video_comments = get_comments_for_video(youtube, video_id)
all_comments.extend(video_comments)

# Create DataFrame
comments_df = pd.DataFrame(all_comments)

# Replace newlines in comments
comments_df['Comment'] = comments_df['Comment'].replace({r'[\r\n]+': ' '}, regex=True)

In [24]:
# Export whole dataset to the local machine as CSV File
csv_file = 'comments_data.csv'  # Name your file
comments_df.to_csv(csv_file, index=False)